In [783]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 20)

In [784]:
hs = pd.read_excel('../data/hs_table_vietnamese.xlsx')
hs.dropna(how='all', inplace=True)

In [785]:
# Create section columns
hs.loc[(hs["CONTENT"].str.lower().str.startswith('section')) & (hs["CONTENT"].str.len() < 20), "SECTION"] = hs.loc[(hs["CONTENT"].str.lower().str.startswith('section')) & (hs["CONTENT"].str.len() < 15), "CONTENT"].str[8:]
hs['TITLE'] = hs['SECTION'].shift(1)
hs.loc[pd.notna(hs['TITLE']), 'TYPE'] = 'SECTION TITLE'
del hs['TITLE']

In [786]:
# Create Chapter columns
hs.loc[(hs["CONTENT"].str.lower().str.startswith('chapter')) & (hs["CONTENT"].str.len() < 12), 'CHAPTER'] = hs.loc[(hs["CONTENT"].str.lower().str.startswith('chapter')) & (hs["CONTENT"].str.len() < 12), 'CONTENT'].str[8:]
hs.loc[pd.notna(hs['CHAPTER']), 'TYPE'] = 'CHAPTER NAME'
hs['TEMP'] = hs['CHAPTER'].shift(1)
hs.loc[pd.notna(hs['TEMP']) & pd.isna(hs['TYPE']), 'TYPE'] = 'CHAPTER TITLE'
del hs['TEMP']

In [787]:
# CREATE HS
hs['HS'] = hs['HS'].astype(str)
hs.loc[hs['HS'].str.len() ==  8, 'TYPE'] = 'HS'

In [788]:
# HEADING AND NOTE OF HEADING
hs.loc[(hs['HS'].str.len() == 4) | ((hs['HS'].str.len() == 8) & (hs['HS'].str.endswith('0000'))), 'HEADING'] = hs.loc[(hs['HS'].str.len() == 4) | ((hs['HS'].str.len() == 8) & (hs['HS'].str.endswith('0000'))), 'HS'].str[:4]
hs.loc[pd.notna(hs['HEADING']), 'TYPE'] = 'HEADING NAME'
hs['TEMP'] = hs['HEADING'].shift(1)
hs.loc[pd.notna(hs['TEMP']) & pd.isna(hs['TYPE']), 'TYPE'] = 'NOTE OF HEADING'

In [789]:
# SUBHEADING
hs.loc[hs['HS'].str.len() == 6, 'SUBHEADING'] = hs.loc[hs['HS'].str.len() == 6, 'HS']
hs.loc[pd.notna(hs['SUBHEADING']) & pd.isna(hs['TYPE']), 'TYPE'] = 'SUBHEADING'

In [790]:



del hs['TEMP']

In [791]:
hs.drop(hs[hs['HS'].str.len() > 8].index, inplace=True)
index_to_delete = hs[(~hs['HS'].str.contains('[0-9]', regex=True, na=True)) & (hs['HS'].str.len() > 3)].index
hs.drop(index_to_delete, inplace=True)


In [792]:
hs.loc[pd.notna(hs['SECTION']), 'TYPE'] = 'SECTION NAME'

In [793]:
hs['TEMP'] = hs['SECTION'].shift(2)
hs.loc[pd.notna(hs['TEMP']) & pd.isna(hs['TYPE']), 'TYPE'] = 'NOTE OF SECTION'
del hs['TEMP']

In [794]:
hs['TEMP'] = hs['CHAPTER'].shift(2)
hs.loc[pd.notna(hs['TEMP']) & pd.isna(hs['TYPE']), 'TYPE'] = 'NOTE OF CHAPTER'
del hs['TEMP']

In [795]:
# DEFINE NOTE OF HS
hs.loc[hs['TYPE'] == 'HS', 'TEMP'] = 'AWAIT'
del hs['DESCRIPTION']
hs['TEMP'] = hs['TEMP'].shift(1)
hs.loc[pd.isna(hs['TYPE']) & pd.notna(hs['TEMP']), 'TYPE'] = 'NOTE OF HS'
del hs['TEMP']



In [796]:
# DEFINE NOTE OF SUBHEADING
hs.loc[hs['TYPE'] == 'SUBHEADING', 'TEMP'] = 'AWAIT'
hs['TEMP'] = hs['TEMP'].shift(1)
hs.loc[pd.isna(hs['TYPE']) & pd.notna(hs['TEMP']), 'TYPE'] = 'NOTE OF SUBHEADING'
del hs['TEMP']

In [797]:

del hs['HEADING']
del hs['SUBHEADING']
del hs['NOTE']

In [798]:
hs.ffill(inplace=True)

In [799]:
hs.drop(hs[((hs['CONTENT'] == 'Notes.') | (hs['CONTENT'] == 'Note.'))].index, inplace=True)



In [800]:
hs['HS'].replace('nan', np.NaN, inplace=True)
hs['HS'].replace(u'\xa0', np.NaN, inplace=True)
hs.ffill(inplace=True)

In [801]:
# hs.to_excel('../data/temp.xlsx')
hs['HEADING'] = hs['HS'].str[:4]



In [802]:
hs['temp'] = hs['HS'].shift(-1)

In [803]:
hs['HS'] = hs['HS'].astype(str)
hs['VIETNAMESE'] = hs['VIETNAMESE'].astype(str)

In [804]:
hs.loc[(hs['TYPE'] == 'NOTE OF HS') & (~hs['VIETNAMESE'].str.startswith('- '))]
hs.drop(hs[(hs['TYPE'] == 'NOTE OF HS') & (~hs['VIETNAMESE'].str.startswith('- '))].index, inplace = True)
hs['temp'] = hs['HS'].shift(-1)
import re

def get_hs5(row):
    if re.match('- ([a-zA-Z0-9])\w+', row['VIETNAMESE']) and len(row['HS']) != 6:
        return row['temp'][0:5]
    else:
        return row['HS']

def format_hs(row):
    if row['HS'].endswith('00'):
        return row['HS'][0: -2]
    else:
        return row['HS']

hs['HS'] = hs.apply(lambda row: get_hs5(row), axis=1)
hs['HS'] = hs.apply(lambda row: format_hs(row), axis=1)

hs.to_excel('lalaland.xlsx')

In [806]:
# if re.match('- ([a-zA-Z0-9])\w+', '- - Ngựa:'):
#     print(True)

In [807]:
hs_dic = []
for _, row in hs.iterrows():
    if row['TYPE'] == 'SECTION NAME':
        hs_dic.append({
            'id': row['SECTION']
        })

    if row['TYPE'] == 'SECTION TITLE':
        hs_dic[-1]['title'] = row['VIETNAMESE']

    # if row['TYPE'] == 'NOTE OF SECTION':
    #     if 'notes' not in hs_dic[-1]:
    #         hs_dic[-1]['notes'] =  []
    #     hs_dic[-1]['notes'].append(row['CONTENT'])


    if row['TYPE'] == 'CHAPTER TITLE':
        if 'chapters' not in hs_dic[-1]:
            hs_dic[-1]['chapters'] =  []
        hs_dic[-1]['chapters'].append({
            'id': row['CHAPTER']
        })
        hs_dic[-1]['chapters'][-1]['title'] = row['VIETNAMESE']

    # if row['TYPE'] == 'NOTE OF CHAPTER':
    #     if 'notes' not in hs_dic[-1]['chapters'][-1]:
    #         hs_dic[-1]['chapters'][-1]['notes'] =  []
    #     hs_dic[-1]['chapters'][-1]['notes'].append(row['CONTENT'])

    if row['TYPE'] == 'HEADING NAME':
        if 'headings' not in hs_dic[-1]['chapters'][-1]:
            hs_dic[-1]['chapters'][-1]['headings'] =  []
        hs_dic[-1]['chapters'][-1]['headings'].append({
            'id': row['HEADING'],
            'title': row['VIETNAMESE']
        })

    # if row['TYPE'] == 'NOTE OF HEADING':
    #     if 'notes' not in hs_dic[-1]['chapters'][-1]['headings'][-1]:
    #         hs_dic[-1]['chapters'][-1]['headings'][-1]['notes'] = []
    #     hs_dic[-1]['chapters'][-1]['headings'][-1]['notes'].append(row['CONTENT'])
    
    
    if row['TYPE'] == 'HS':
        if 'hs' not in hs_dic[-1]['chapters'][-1]['headings'][-1]:
            hs_dic[-1]['chapters'][-1]['headings'][-1]['hs'] =  []
        hs_dic[-1]['chapters'][-1]['headings'][-1]['hs'].append({
            'id': row['HS'],
            'title': row['VIETNAMESE']
        })
    
    # if row['TYPE'] == 'NOTE OF HS':
    #     if 'notes' not in hs_dic[-1]['chapters'][-1]['headings'][-1]['hs'][-1]:
    #         hs_dic[-1]['chapters'][-1]['headings'][-1]['hs'][-1]['notes'] = []
    #     hs_dic[-1]['chapters'][-1]['headings'][-1]['hs'][-1]['notes'].append(row['CONTENT'])

    # if row['TYPE'] == 'SUBHEADING':
    #     if 'hs' not in hs_dic[-1]['chapters'][-1]['headings'][-1]:
    #         hs_dic[-1]['chapters'][-1]['headings'][-1]['hs'] =  []
    #     hs_dic[-1]['chapters'][-1]['headings'][-1]['hs'].append({
    #         'id': row['HS'],
    #         'title': row['CONTENT']
    #     })
    
    # if row['TYPE'] == 'NOTE OF SUBHEADING':
    #     if 'notes' not in hs_dic[-1]['chapters'][-1]['headings'][-1]['hs'][-1]:
    #         hs_dic[-1]['chapters'][-1]['headings'][-1]['hs'][-1]['notes'] = []
    #     hs_dic[-1]['chapters'][-1]['headings'][-1]['hs'][-1]['notes'].append(row['CONTENT'])

hs.to_excel('laland.xlsx')

import json
with open('data.json', 'w') as f:
    json.dump(hs_dic, f)

In [808]:
with open('vi_data2.json') as json_file:
    data = json.load(json_file)

data.pop()
print('lol')

lol


In [809]:
# hs.loc[hs['HS'] == '9705', ['VIETNAMESE']].iloc[0]['VIETNAMESE']

In [810]:
data[0]['chapters'][0]['headings'][0]['hs6'][0]

for section in data:
    for chapter in section['chapters']:
        for heading in chapter['headings']:
            for hs6 in heading['hs6']:
                try:
                    hs6['title'] = hs.loc[(hs['HS']) == ''.join(hs6['id'].split('.')), ['VIETNAMESE']].iloc[0]['VIETNAMESE']
                except IndexError:
                    print(''.join(hs6['id'].split('.')))

01012
010121
010129
010190
01022
010221
0102291
01023
010231
010239
010310
01039
010391
010392
01051
01059
0105944
0105949
01061
010611
010612
010613
010614
010619
010620
01063
010631
010632
010633
010639
01064
010641
010649
010690
020110
020120
020130
020210
020220
020230
02031
020311
020312
020319
02032
020321
020322
020329
020410
02042
020421
020422
020423
020430
02044
020441
020442
020443
020450
020610
02062
020621
020622
020629
020630
02064
020641
020649
020680
020690
02071
020711
020712
020713
0207149
02072
020724
020725
020726
0207279
02074
020741
020742
020743
020744
020745
02075
020751
020752
020753
020754
020755
020760
020810
020830
020850
020860
020910
020990
02101
021011
021012
021020
02109
021091
021093
03011
0301111
0301119
0301199
03019
030191
030192
030194
030195
0301991
0301992
0301994
0301995
03021
030211
030213
030214
030219
03022
030221
030222
030223
030224
030229
03023
030231
030232
030233
030234
030235
030236
030239
03024
030241
030242
030243
030244
030245
030246


In [820]:
bst = pd.read_excel('BIEU-THUE-XNK-2021.06.22.xlsx', header=4, converters={'B':str})

In [830]:
bst = bst[['B', 'C', 'D']]
bst['C'] = bst['C'].astype(str)
bst['D'] = bst['D'].astype(str)

d:\Projects\Excel-to-SQL-snippets\env\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [834]:
bst['temp'] = bst['B'].shift(-1)
bst['temp'] = bst['temp'].astype(str)

In [837]:
bst.drop(bst[(bst['B'] == 'nan') & (bst['C'] == 'nan')].index, inplace = True)

In [838]:
def get_hs5(row):
    if re.match('- ([a-zA-Z0-9])\w+', row['C']) and pd.isna(row['B']):
        print(row['temp'][0:5])
        return row['temp'][0:5]
    else:
        return row['B']

bst['B'] = bst.apply(lambda row: get_hs5(row), axis=1)

01012
01022
01023
01039
01051
01059
01063
01064
02031
02042
02044
02062
02064
02071
02072
02074
02075
02101
02109
03011
03019
03021
03022
03023
03024
03025
03027
nan
03029
03031
03032
03033
03034
03035
03036
03038
03039
03043
03044
03045
03046
03047
03048
03049
03053
03054
03055
03056
03057
03063
03069
03071
03073
03074
03075
03077
03078
03079
03081
03082
04022
04029
04071
04072
04081
04089
05119
06031
07051
07052
07095
07099
07102
07115
07123
08011
08012
08013
08021
08022
08023
08024
08025
08026
08052
08071
08092
09011
09012
09041
09042
09061
09081
09082
09083
09092
09093
09096
09101
09109
10011
10019
10082
11010
11031
11041
11042
11081
12024
12072
12079
12092
12099
12122
12129
13021
13023
15121
15122
15131
15132
15141
15149
15151
15152
15180
15180
16023
16024
16041
16043
16052
17019
17021
18063
19021
19053
20059
20079
20081
20089
20091
20092
20093
20094
20096
20097
20098
21011
22029
22030
22030
22042
22060
22060
22060
23064
24031
24039
25010
25161
25232
25292
26011
26201
26202
26209


In [ ]:
hs.loc[(hs['TYPE'] == 'NOTE OF HS') & (~hs['VIETNAMESE'].str.startswith('- '))]
hs.drop(hs[(hs['TYPE'] == 'NOTE OF HS') & (~hs['VIETNAMESE'].str.startswith('- '))].index, inplace = True)
hs['temp'] = hs['HS'].shift(-1)
import re

def get_hs5(row):
    if re.match('- ([a-zA-Z0-9])\w+', row['VIETNAMESE']) and len(row['HS']) != 6:
        return row['temp'][0:5]
    else:
        return row['HS']

def format_hs(row):
    if row['HS'].endswith('00'):
        return row['HS'][0: -2]
    else:
        return row['HS']

hs['HS'] = hs.apply(lambda row: get_hs5(row), axis=1)
hs['HS'] = hs.apply(lambda row: format_hs(row), axis=1)

hs.to_excel('lalaland.xlsx')